In [3]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from PIL import Image
import matplotlib.pyplot as plt
from swin_functions_and_classes import *

from going_modular.going_modular import engine
from helper_functions import set_seeds
import pickle



current_directory = os.getcwd()

In [2]:
# !pip install datasets

In [5]:
from datasets import load_dataset
current_directory = os.getcwd()
train_dir = os.path.join(current_directory, "Data", "Train")
test_dir = os.path.join(current_directory, "Data", "Test")

ds = load_dataset(train_dir, split="train", trust_remote_code=True)

Resolving data files:   0%|          | 0/1220 [00:00<?, ?it/s]

In [9]:
ds.features['label']

ClassLabel(names=['COVID', 'NORMAL', 'PNEUMONIA'], id=None)

In [10]:
ds2 = load_dataset('food101')
ds2

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})

In [12]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1220
})

In [13]:
labels = ds.features['label']
print(labels)

# getting label of our example
ex = ds[400]
print(labels.int2str(ex['label']))

ClassLabel(names=['COVID', 'NORMAL', 'PNEUMONIA'], id=None)
NORMAL


In [16]:
# !pip install transformers

In [17]:
from transformers import AutoFeatureExtractor

#loading the feature extractor
model_name= 'microsoft/swin-base-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs
  
# applying transform
prepared_ds = ds.with_transform(transform)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

c:\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Iman\.cache\huggingface\hub\models--microsoft--swin-base-patch4-window7-224. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
